# A questão dos dados: análises iniciais e preparação

### Importar os dados já armazenados

In [1]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/rlrocha90/IAeML-Treinamento/main/housing.csv'

housing = pd.read_csv(url, index_col=0)
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

<class 'NameError'>: name 'pd' is not defined

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

### Apresentar uma tabela com amostra dos dados

In [ ]:
print(housing.head())

### Apresenta informações importantes sobre os dados

In [ ]:
print(housing.info())

### Apresentar contagem das informações dentro da coluna "ocean_proximity", por conter objetos

In [ ]:
print(housing["ocean_proximity"].value_counts())

### Apresenta dados estatísticos sobre o conjunto

In [ ]:
print(housing.describe())
statt = housing.describe()

### Apresenta um histograma

In [ ]:
import matplotlib.pyplot as plt

housing.hist(bins=50, figsize=(20, 15))
plt.show()

### Matriz de Correlação

In [ ]:
corr_matrix = housing.corr()

### Apresenta as correlações do atributo "median_house_value" com os demais do conjunto

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

### Apresenta um conjunto de gráficos com a relação entre duas variáveis...
### É preciso escolher as variáveis...

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])
plt.show()

### Algumas manipulações - FEATURE ENGINEERING

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

### Analisar mais uma vez a correlação, agora com as novas features

In [ ]:
corr_matrix = housing.corr()
print(corr_matrix["median_house_value"].sort_values(ascending=False))

attributes = ["median_house_value", "bedrooms_per_room"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

### Mais uma olhada nos dados

In [ ]:
housing.describe()

### Extrair de housing a informação das labels, somente X

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)

### Criar um novo conjunto com as labels, somente y

In [ ]:
housing_labels = strat_train_set["median_house_value"].copy()

### Exemplos de limpeza dos dados
### Verificiar valores incompletos

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
print(sample_incomplete_rows)

### Podemos tomar algumas ações
### Opção 1 - drop

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])

### Opção 2 - drop

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)

### Opção 3 - Preenchimento com valores

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True)
print(sample_incomplete_rows)

### Tratando com atributos categóricos e textos

In [ ]:
housing_cat = housing[['ocean_proximity']]
print(housing_cat.head(10))

### Codificando os atributos em valores numéricos

In [ ]:
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
print(housing_cat_encoded[:10])
print(ordinal_encoder.categories_)

### Outro método, usando representação binária

In [ ]:
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
print(housing_cat_1hot)
print(housing_cat_1hot.toarray())